In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import requests
import time

from tqdm import tqdm_notebook

## 알라딘 서점 데이터 웹크롤링

In [7]:
# 자동화 웹 페이지 열기
path = './driver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="5ac34733f83851a425cb7c134f2f032c")>

In [85]:
# 사회과학 분야 url
category_url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=798&Year=2020&Month='
rest_url01 = '&page='
rest_url02 = '&cnt=1000&SortOrder=1'

In [118]:
# 책 제목 추출을 위한 단어 리스트
file_format = ['(지은이)', '(옮긴이)', '(엮은이)', '(글)', '(그림)', '(편곡)', '(감수)', '(원작)', '(참고서)', '(캐릭터)', '(기획)', '(구성)', '(디자인)', '(해설)', '김은숙, 김수연']

# 저자 및 출판사를 담을 list
book_list = []

image =[]
title = []
isbn_list = []

In [119]:
# 1 ~ 10월
# 각 월별 1위 ~ 250위 데이터 추출

cnt = 0

# 월별 페이지 이동
for month in tqdm_notebook(range(5, 11)) :
    
    # 한 월별 6개의 페이지 접근 (1위 ~ 250위)
    for page in range(1, 6) :
        url = category_url + str(month) + rest_url01 + str(page) + rest_url02
        driver.get(url)
        
        # 필요한 데이터 부분 크롤링
        list = driver.find_elements_by_css_selector("div.ss_book_box > table > tbody > tr")
        
        req = requests.get(url)
        content = req.content
        
        # 크롤링 데이터를 html 형식에 맞게 변환
        soup = BeautifulSoup(content,'html.parser')
        book_name_elems = soup.select('a.bo3 b')
        
        # 이미지 url, 제목 추출
        # 저자, 출판사가 포함되어있는 문자열 추출
        for i in range(len(list)) :
            split_ls = list[i].text.split('\n')
            
            image_ls = list[i].find_element_by_tag_name('img').get_attribute('src')
            image.append(image_ls)
            
            title.append(book_name_elems[i].get_text())

            for j in split_ls :
                if (file_format[0] in j) | (file_format[1] in j) | (file_format[2] in j) | (file_format[3] in j) | (file_format[4] in j) | (file_format[5] in j) | (file_format[6] in j) | (file_format[7] in j) | (file_format[8] in j) | (file_format[9] in j) | (file_format[10] in j) | (file_format[11] in j) | (file_format[12] in j) | (file_format[13] in j) | (file_format[14] in j) :
                    book_list.append(j)
                    cnt += 1
                    break
        
        print('{}월 {}페이지 cnt = {}'.format(month, page, cnt))
        cnt = 0
        
        
        # isbn 크롤링을 위해 페이지 이동 구현
        # 링크 리스트 생성
        link_list = driver.find_elements_by_css_selector('a.bo3')
        
        for idx in tqdm_notebook(range(len(link_list))) :
            checkFlag = False
            link_list[idx].click()
            
            isbn =  driver.find_elements_by_css_selector("div.conts_info_list1 > ul > li")
            
            for li in isbn :
                if 'ISBN' in li.text :
                    isbn_list.append(li.text.split(':')[1].strip())
                    checkFlag = True
                    break
            
            if checkFlag is not True :
                isbn_list.append(np.nan)
            
            driver.back()
            link_list = driver.find_elements_by_css_selector('a.bo3')
            
            time.sleep(3)
            link_list = driver.find_elements_by_css_selector('a.bo3')
            
        print(len(isbn_list))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


5월 1페이지 cnt = 50


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



50
5월 2페이지 cnt = 50



100
5월 3페이지 cnt = 50


...


1300
10월 2페이지 cnt = 50



1350
10월 3페이지 cnt = 50



1400
10월 4페이지 cnt = 50



1450
10월 5페이지 cnt = 50



1500



In [121]:
# 전체 길이 확인
print(len(isbn_list))
print(len(title))
print(len(image))
print(len(book_list))

1500
1500
1500
1500


In [122]:
# 작가, 출판사 가져오기
split_booklist=[]
author = []
publisher =[]

for s in range(len(book_list)) :
    input = book_list[s].split('|')
    split_booklist.append(input)

    author.append(split_booklist[s][0].strip())
    publisher.append(split_booklist[s][1].strip())

In [123]:
# 확인
print(split_booklist[0:5])
print(len(split_booklist))
print()
print(author[0:5])
print(len(author))
print()
print(publisher[0:5])
print(len(publisher))

[['팀 마샬 (지은이), 김미선 (옮긴이) ', ' 사이 ', ' 2016년 8월'], ['이수정, 이다혜, 최세희, 조영주 (지은이) ', ' 민음사 ', ' 2020년 3월'], ['조정진 (지은이) ', ' 후마니타스 ', ' 2020년 3월'], ['말콤 글래드웰 (지은이), 유강은 (옮긴이), 김경일 (감수) ', ' 김영사 ', ' 2020년 3월'], ['권사랑, 서한나, 이민경 (지은이) ', ' BOSHU ', ' 2020년 4월']]
1500

['팀 마샬 (지은이), 김미선 (옮긴이)', '이수정, 이다혜, 최세희, 조영주 (지은이)', '조정진 (지은이)', '말콤 글래드웰 (지은이), 유강은 (옮긴이), 김경일 (감수)', '권사랑, 서한나, 이민경 (지은이)']
1500

['사이', '민음사', '후마니타스', '김영사', 'BOSHU']
1500


In [124]:
# 데이터 프레임 만들기
book_5to10 = pd.DataFrame({
    'title' : title,
    'author' : author,
    'publisher' : publisher,
    'isbn' : isbn_list,
    'image' : image,
})

In [108]:
# csv파일로 데이터프레임 저장
book.to_csv('social_science.csv', mode = 'w', index =  False, encoding = 'utf-8-sig')

In [117]:
book_2to4.to_csv('social_science_2to4.csv', mode = 'w', index =  False, encoding = 'utf-8-sig')

In [127]:
book_5to10.to_csv('social_science_5to10.csv', mode = 'w', index =  False, encoding = 'utf-8-sig')

In [131]:
# 데이터프레임 통합 작업
social_science_df = pd.concat([book, book_2to4, book_5to10], ignore_index = True)

In [132]:
social_science_df.shape

(2500, 5)

In [133]:
# csv 파일로 데이터프레임 저장
social_science_df.to_csv('social_science_df.csv', mode = 'w', index = False, encoding = 'utf-8-sig')

In [136]:
len(social_science_df['isbn'].unique())

1002

In [137]:
# isbn을 기준으로 중복 데이터 제거
social_science_df_remove = social_science_df.drop_duplicates('isbn', keep = 'first')

In [138]:
social_science_df_remove.shape

(1002, 5)

In [139]:
# csv 파일로 데이터프레임 저장
social_science_df_remove.to_csv('social_science_df_remove.csv', mode = 'w', index = False, encoding = 'utf-8-sig')

## 알라딘 서점 데이터 웹크롤링 (19세 이하 조회 불가 도서 제외) - 소설 / 장르소설

In [32]:
# 소설/시/장르 url
category_url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=1&Year=2020&Month='
rest_url01 = '&page='
rest_url02 = '&cnt=1000&SortOrder=1'

In [9]:
# 책 제목 추출을 위한 단어 리스트
file_format = ['(지은이)', '(옮긴이)', '(엮은이)', '(글)', '(그림)', '(편곡)', '(감수)', '(원작)', '(참고서)', '(캐릭터)', '(기획)', '(구성)', '(디자인)', '(해설)', '김은숙, 김수연']

# 저자 및 출판사를 담을 list
book_list = []

image =[]
title = []
isbn_list = []

In [34]:
# 1 ~ 10월
# 각 월별 1위 ~ 250위 데이터 추출
# 총 2500권의 책 데이터 추출

# 책 제목 추출을 위한 단어 리스트
file_format = ['(지은이)', '(옮긴이)', '(엮은이)', '(글)', '(그림)', '(편곡)', '(감수)', '(원작)', '(참고서)', '(캐릭터)', '(기획)', '(구성)', '(디자인)', '(해설)', '김은숙, 김수연']

# 저자 및 출판사를 담을 list
book_list = []

# 데이터를 담을 list
image =[]
title = []
isbn_list = []

# 19금 책 데이터의 인덱스를 담을 list
ban = []

# 데이터가 모두 잘 들어왔는지 확인하는 debug 변수
cnt = 0


# 월별 페이지 이동
for month in tqdm_notebook(range(1, 11)) :
    
    # 한 월별 6개의 페이지 접근 (1위 ~ 250위)
    for page in range(1, 6) :
        url = category_url + str(month) + rest_url01 + str(page) + rest_url02
        driver.get(url)
        
        # 필요한 데이터 부분 크롤링
        list = driver.find_elements_by_css_selector("div.ss_book_box > table > tbody > tr")
        
        req = requests.get(url)
        content = req.content
        
        # 크롤링 데이터를 html 형식에 맞게 변환
        soup = BeautifulSoup(content,'html.parser')
        book_name_elems = soup.select('a.bo3 b')
        
        # 이미지 url, 제목 추출
        # 저자, 출판사가 포함되어있는 문자열 추출
        for i in range(len(list)) :
            split_ls = list[i].text.split('\n')
            
            image_ls = list[i].find_elements_by_tag_name('img')
            
            # 이미지를 크롤링했을 때 가장 처음 나오는 이미지가 19세 이하 조회 불가 이미지라면
            # 인덱스만 저장하고 넘어간다.
            if '19_150cover.png' in image_ls[0].get_attribute('src') :
                ban.append(i)
                continue
            else :
                image.append(image_ls[0].get_attribute('src'))
            
            title.append(book_name_elems[i].get_text())

            for j in split_ls :
                if (file_format[0] in j) | (file_format[1] in j) | (file_format[2] in j) | (file_format[3] in j) | (file_format[4] in j) | (file_format[5] in j) | (file_format[6] in j) | (file_format[7] in j) | (file_format[8] in j) | (file_format[9] in j) | (file_format[10] in j) | (file_format[11] in j) | (file_format[12] in j) | (file_format[13] in j) | (file_format[14] in j) :
                    book_list.append(j)
                    cnt += 1
                    break
        
        # debug
        print('{}월 {}페이지 cnt = {}'.format(month, page, cnt))
        print('ban       length :', len(ban))
        print('image     length :', len(image))
        print('title     length :', len(title))
        print('book_list length :', len(book_list))
        cnt = 0
        
        
        # isbn 크롤링을 위해 페이지 이동 구현
        # 링크 리스트 생성
        link_list = driver.find_elements_by_css_selector('a.bo3')
        
        for idx in tqdm_notebook(range(len(link_list))) :
            checkFlag = False
            
            # 19세 이하 조회 불가 도서의 인덱스 차례라면
            # 클릭을 하지 않고 넘어간다. 
            if idx in ban :
                continue
            else :
                link_list[idx].click()
            
            isbn =  driver.find_elements_by_css_selector("div.conts_info_list1 > ul > li")
            
            for li in isbn :
                if 'ISBN' in li.text :
                    isbn_list.append(li.text.split(':')[1].strip())
                    checkFlag = True
                    break
            
            if checkFlag is not True :
                isbn_list.append(np.nan)
            
            driver.back()
            link_list = driver.find_elements_by_css_selector('a.bo3')
            
            time.sleep(3)
            link_list = driver.find_elements_by_css_selector('a.bo3')
            
        print(len(isbn_list))
        ban = []

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


1월 1페이지 cnt = 47
ban       length : 3
image     length : 47
title     length : 47
book_list length : 47


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



47
1월 2페이지 cnt = 50
ban       length : 0
image     length : 97
title     length : 97
book_list length : 97



97
1월 3페이지 cnt = 50
ban       length : 0
image     length : 147
title     length : 147
book_list length : 147


...


2282
10월 2페이지 cnt = 49
ban       length : 1
image     length : 2331
title     length : 2331
book_list length : 2331



2331
10월 3페이지 cnt = 49
ban       length : 1
image     length : 2380
title     length : 2380
book_list length : 2380



2380
10월 4페이지 cnt = 50
ban       length : 0
image     length : 2430
title     length : 2430
book_list length : 2430



2430
10월 5페이지 cnt = 50
ban       length : 0
image     length : 2480
title     length : 2480
book_list length : 2480



2480



In [35]:
# 전체 길이 확인
print(len(isbn_list))
print(len(title))
print(len(image))
print(len(book_list))

2480
2480
2480
2480


In [36]:
# 작가, 출판사 가져오기
split_booklist=[]
author = []
publisher =[]

for s in range(len(book_list)) :
    input = book_list[s].split('|')
    split_booklist.append(input)

    author.append(split_booklist[s][0].strip())
    publisher.append(split_booklist[s][1].strip())

In [37]:
# 확인
print(split_booklist[0:5])
print(len(split_booklist))
print()
print(author[0:5])
print(len(author))
print()
print(publisher[0:5])
print(len(publisher))

[['나태주 (지은이) ', ' 열림원 ', ' 2019년 12월'], ['김초엽 (지은이) ', ' 허블 ', ' 2019년 6월'], ['묵향동후 (지은이), 이현아 (옮긴이) ', ' B-Lab ', ' 2019년 7월'], ['장류진 (지은이) ', ' 창비 ', ' 2019년 10월'], ['묵향동후 (지은이), 이현아 (옮긴이) ', ' B-Lab ', ' 2019년 11월']]
2480

['나태주 (지은이)', '김초엽 (지은이)', '묵향동후 (지은이), 이현아 (옮긴이)', '장류진 (지은이)', '묵향동후 (지은이), 이현아 (옮긴이)']
2480

['열림원', '허블', 'B-Lab', '창비', 'B-Lab']
2480


In [38]:
# 데이터 프레임 만들기
book = pd.DataFrame({
    'title' : title,
    'author' : author,
    'publisher' : publisher,
    'isbn' : isbn_list,
    'image' : image,
})

In [39]:
book.head()

,title,author,publisher,isbn,image
0,너와 함께라면 인생도 여행이다,나태주 (지은이),열림원,9791170400097,https://image.aladin.co.kr/product/22160/14/co...
1,우리가 빛의 속도로 갈 수 없다면,김초엽 (지은이),허블,9791190090018,https://image.aladin.co.kr/product/19359/16/co...
2,마도조사 1,"묵향동후 (지은이), 이현아 (옮긴이)",B-Lab,9791127851446,https://image.aladin.co.kr/product/19904/52/co...
3,일의 기쁨과 슬픔,장류진 (지은이),창비,9788936438036,https://image.aladin.co.kr/product/21292/13/co...
4,마도조사 3,"묵향동후 (지은이), 이현아 (옮긴이)",B-Lab,9791127853068,https://image.aladin.co.kr/product/21172/79/co...


In [40]:
book.tail()

,title,author,publisher,isbn,image
2475,당신 인생의 이야기,"테드 창 (지은이), 김상훈 (옮긴이)",엘리,9788956057842,https://image.aladin.co.kr/product/9401/86/cov...
2476,침묵,"돈 드릴로 (지은이), 송은주 (옮긴이)",창비,9788936478230,https://image.aladin.co.kr/product/25324/79/co...
2477,이데아,이유월 (지은이),필프리미엄에디션(FEEL),9791165654948,https://image.aladin.co.kr/product/25226/22/co...
2478,침묵 박물관,"오가와 요코 (지은이), 이윤정 (옮긴이)",작가정신,9791160261943,https://image.aladin.co.kr/product/25159/23/co...
2479,바쇼의 하이쿠,"마쓰오 바쇼 (지은이), 유옥희 (옮긴이)",민음사,9788937475351,https://image.aladin.co.kr/product/25058/98/co...


In [41]:
# csv로 데이터프레임 저장
book.to_csv('fiction_poem_df.csv', mode = 'w', index = True, encoding = 'utf-8-sig')

In [43]:
len(book['isbn'].unique())

1020

In [44]:
# isbn을 기준으로 데이터 중복 제거
book_remove = book.drop_duplicates('isbn', keep = 'first')
book_remove.shape

(1020, 5)

In [45]:
# csv로 데이터프레임 저장
book_remove.to_csv('fiction_poem_df_remove.csv', mode = 'w', index = False, encoding = 'utf-8-sig')

## 교보문고 웹페이지에서 isbn을 이용한 책의 키워드 추출

In [146]:
# 서점 데이터 불러오기
book_remove_df = pd.read_csv('book_remove_df.csv')

In [148]:
# isbn 리스트 생성
isbn_list = book_remove_df['isbn']

In [194]:
type(isbn_list[0])

numpy.int64

In [142]:
len(isbn_list)

6903

In [128]:
# 접근할 url 변수 생성
isbn_url = 'http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode='
rest_url = '&orderClick=LET&Kc='

In [129]:
# 자동화 웹 생성
path = './driver/chromedriver.exe'
driver = webdriver.Chrome(path)

In [192]:
# 데이터가 모두 잘 들어왔는지 확인하는 debug 변수
cnt = 0
keyword = []
keyword_list = []

# 월별 페이지 이동
for i in tqdm_notebook(range(5250, len(isbn_list))):
    url = isbn_url + str(isbn_list[i]) + rest_url
    driver.get(url)
    
    a_list = driver.find_elements_by_css_selector("div.book_keyword >a")
    time.sleep(1.5)
    
    a_list = driver.find_elements_by_css_selector("div.book_keyword >a")
    print('list length :', len(a_list))
    
    for i in range(len(a_list)):
        pick = a_list[i].text
        cnt += 1
        keyword.append(pick)
    
    print('pick(cnt) length :', cnt)
    print('keyword :', keyword)
    keyword_list.append(keyword)
    
    print('keyword_list length :', len(keyword_list))
    keyword = []
    
    print()
    cnt = 0

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


list length : 6
pick(cnt) length : 6
keyword : ['현대 연극', '세기 러시아 문학', '세계 문학사', '단막극', '인내', '고리키']
keyword_list length : 5251

list length : 6
pick(cnt) length : 6
keyword : ['음악', '그림', '기악', '하이든', '예술 분야', '바흐']
keyword_list length : 5252

list length : 6
pick(cnt) length : 6
keyword : ['노랫말', '가요', '작사', '시인', '명반', '가시나무']
keyword_list length : 5253

list length : 6
pick(cnt) length : 6
keyword : ['심리학', '병리학', '잠재의식', '탐지', '공학', '일상 용품']
keyword_list length : 5254

list length : 6
pick(cnt) length : 6
keyword : ['느와르', '생태학', '이주자', '폭력', '동시대', '미학']
keyword_list length : 5255

list length : 6
pick(cnt) length : 6
keyword : ['악보', '알파벳', '음자리표', '화음', '대위법', '음악사']
keyword_list length : 5256

list length : 6
pick(cnt) length : 6
keyword : ['퓰리처상', '극락', '인물', '파멸', '미치', '희곡집']
keyword_list length : 5257

list length : 6
pick(cnt) length : 6
keyword : ['미술사', '아방가르드', '모나리자', '미켈란젤로', '비너스', '모더니즘']
keyword_list length : 5258

list length : 6
pick(cnt) length : 6
keyword : ['슬

list length : 6
pick(cnt) length : 6
keyword : ['포르', '레르', '준비', '심화', '스트레칭', '이동과']
keyword_list length : 5323

list length : 6
pick(cnt) length : 6
keyword : ['두려움', '복음서', '예술', '중도', '오선지', '자율 신경']
keyword_list length : 5324

list length : 6
pick(cnt) length : 6
keyword : ['연주자', '독일소설', '작품', '희곡', '독일어', '독일문학']
keyword_list length : 5325

list length : 6
pick(cnt) length : 6
keyword : ['미술사', '화가', '베스트셀러', '문화 예술', '르누아르', '시리즈']
keyword_list length : 5326

list length : 6
pick(cnt) length : 6
keyword : ['웹툰', '그래픽 노블', '코믹', '스토리텔링', '만화가', '몸짓 언어']
keyword_list length : 5327

list length : 6
pick(cnt) length : 6
keyword : ['작법', '디자인', '파트', '입 문자', '그리기', '기법']
keyword_list length : 5328

list length : 6
pick(cnt) length : 6
keyword : ['열하일기', '만보', '최승희', '요정', '동아', '고생물학']
keyword_list length : 5329

list length : 6
pick(cnt) length : 6
keyword : ['통사', '고전주의', '리바이벌', '중세 문화', '알베르티', '오라토리오']
keyword_list length : 5330

list length : 6
pick(cnt) length : 6
keyword : 

list length : 6
pick(cnt) length : 6
keyword : ['도안', '캘리그라피', '선물', '긍정', '수채화', '행복']
keyword_list length : 5394

list length : 6
pick(cnt) length : 6
keyword : ['미술', '예술가', '테일러', '미술관', '히치콕', '비엔날레']
keyword_list length : 5395

list length : 6
pick(cnt) length : 6
keyword : ['한국에세이', '그림에세이', '기분', '여행', '가게', '제주']
keyword_list length : 5396

list length : 6
pick(cnt) length : 6
keyword : ['화가', '국민', '이케아', '스웨덴', '나스', '히트']
keyword_list length : 5397

list length : 0
pick(cnt) length : 0
keyword : []
keyword_list length : 5398

list length : 6
pick(cnt) length : 6
keyword : ['일본라이트노벨', '일러스트', '일러스트레이터', '스케치', '캐릭터 디자인', '파트']
keyword_list length : 5399

list length : 6
pick(cnt) length : 6
keyword : ['게임', '엘리', '조엘', '스토리 라인', '생존자', '파트']
keyword_list length : 5400

list length : 6
pick(cnt) length : 6
keyword : ['피아노', '전설', '그랜드', '콘서트', '뉴욕 타임스', '이민자']
keyword_list length : 5401

list length : 6
pick(cnt) length : 6
keyword : ['팝송', '음반', '가요', '증언', '흑역사', '대중 음악']
k

list length : 6
pick(cnt) length : 6
keyword : ['공간', '자연', '코펜하겐', '타이포', '미학', '스튜디오']
keyword_list length : 5465

list length : 6
pick(cnt) length : 6
keyword : ['다이어리', '컬러링', '색연필', '베스트셀러', '도안', '스티커']
keyword_list length : 5466

list length : 6
pick(cnt) length : 6
keyword : ['테크닉', '만화작법', '첨삭 해설', '연출', '만화가', '강좌']
keyword_list length : 5467

list length : 6
pick(cnt) length : 6
keyword : ['한국에세이', '위로', '그림에세이', '미술이야기', '인생', '방송인']
keyword_list length : 5468

list length : 6
pick(cnt) length : 6
keyword : ['사무라이', '닌자', '계승자', '전국 시대', '섀도우', '천애']
keyword_list length : 5469

list length : 6
pick(cnt) length : 6
keyword : ['혁신', '사조', '모더니즘', '입문서', '인상주의', '현대미술사']
keyword_list length : 5470

list length : 6
pick(cnt) length : 6
keyword : ['해설', '미술사', '유튜브', '어른', '질문', '르네상스 시대']
keyword_list length : 5471

list length : 6
pick(cnt) length : 6
keyword : ['미술이야기', '로마 신화', '상상력', '예술', '미술가', '미술 작품']
keyword_list length : 5472

list length : 6
pick(cnt) length : 6
keyw

...

list length : 6
pick(cnt) length : 6
keyword : ['창업', '모바일', '재무 관리', '소자본', '노후 준비', '금리']
keyword_list length : 6596

list length : 6
pick(cnt) length : 6
keyword : ['행복', '삶의지혜', '스마트폰', '인생', '넷플릭스', '나무늘보']
keyword_list length : 6597

list length : 6
pick(cnt) length : 6
keyword : ['공감', '심리학자', '일본', '심리', '남녀 관계', '남녀심리']
keyword_list length : 6598

list length : 6
pick(cnt) length : 6
keyword : ['심리치료', '아마존', '토킹', '크레이지', '인사이트', '스스로']
keyword_list length : 6599

list length : 6
pick(cnt) length : 6
keyword : ['변화', '헤어', '긴자', '여성처세', '에세', '선택']
keyword_list length : 6600

list length : 6
pick(cnt) length : 6
keyword : ['사랑', '심리학', '레이첼', '애착', '모나코', '허먼']
keyword_list length : 6601

list length : 6
pick(cnt) length : 6
keyword : ['철학', '마음', '작법', '문장', '아사히 신문', '단어']
keyword_list length : 6602

list length : 6
pick(cnt) length : 6
keyword : ['행복', '스마트폰', '내면', '가해자', '대화법', '요구사항']
keyword_list length : 6603

list length : 6
pick(cnt) length : 6
keyword : ['인간관계', '건

list length : 6
pick(cnt) length : 6
keyword : ['판타지소설', '판타지', '성장소설', '청소년', '인간', '장르 소설']
keyword_list length : 6667

list length : 0
pick(cnt) length : 0
keyword : []
keyword_list length : 6668

list length : 6
pick(cnt) length : 6
keyword : ['판타지소설', '영화원작소설', '성장소설', '영미소설', '마법 학교', '영국소설']
keyword_list length : 6669

list length : 6
pick(cnt) length : 6
keyword : ['판타지소설', '영화원작소설', '성장소설', '영미소설', '고전', '소설']
keyword_list length : 6670

list length : 6
pick(cnt) length : 6
keyword : ['판타지', '넷플릭스', '시리', '드래곤', '검술', '골렘']
keyword_list length : 6671

list length : 6
pick(cnt) length : 6
keyword : ['작가', '추리', '탐정', '주부', '소희', '범인']
keyword_list length : 6672

list length : 6
pick(cnt) length : 6
keyword : ['살인사건', '오름', '베리', '범죄 소설', '스웨덴소설', '미스터리스릴러']
keyword_list length : 6673

list length : 6
pick(cnt) length : 6
keyword : ['한국 소설', '판타지소설', '캐릭터', '카카오 페이지', '공녀', '하이판타지']
keyword_list length : 6674

list length : 6
pick(cnt) length : 6
keyword : ['소설', '목소리', '유령', '공

list length : 6
pick(cnt) length : 6
keyword : ['미스터리소설', '영미소설', '드라마원작소설', '살인사건', '레이스', '하녀']
keyword_list length : 6736

list length : 6
pick(cnt) length : 6
keyword : ['판타지소설', '시인', '해리 포터', '신비', '비들', '단장']
keyword_list length : 6737

list length : 6
pick(cnt) length : 6
keyword : ['시간 도둑', '환상', '회색', '시간', '신사', '문학 이야기']
keyword_list length : 6738

list length : 6
pick(cnt) length : 6
keyword : ['청소년', '호재', '외계인', '슬리퍼', '베타', '지구']
keyword_list length : 6739

list length : 6
pick(cnt) length : 6
keyword : ['미스터리', '우정', '여성', '범죄 수사', '시신', '코지']
keyword_list length : 6740

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '로맨스소설', '웹소설', '이사', '싸가지', '여자 주인공']
keyword_list length : 6741

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '사랑', '로맨스소설', '소설', '웹소설', '연애']
keyword_list length : 6742

list length : 6
pick(cnt) length : 6
keyword : ['미스터리소설', '미스터리', '서원', '정진', '아내', '애인']
keyword_list length : 6743

list length : 6
pick(cnt) length : 6
keyword :

list length : 6
pick(cnt) length : 6
keyword : ['스릴러소설', '프랑스소설', '르포', '폭력', '암살', '누아르']
keyword_list length : 6806

list length : 6
pick(cnt) length : 6
keyword : ['스릴러소설', '영미소설', '미국소설', '심리 스릴러', '서스펜스', '여자']
keyword_list length : 6807

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '로맨스소설', '송하', '맨투맨', '우제', '카풀']
keyword_list length : 6808

list length : 6
pick(cnt) length : 6
keyword : ['사랑', '소설', '성장소설', '첫사랑', '로맨스', '소녀']
keyword_list length : 6809

list length : 6
pick(cnt) length : 6
keyword : ['추리', '형사', '전직', '인터넷 속', '로맨틱', '낙원']
keyword_list length : 6810

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '소설', '미스터리', '웹소설', '스릴러', '가족']
keyword_list length : 6811

list length : 6
pick(cnt) length : 6
keyword : ['고양이', '고등 학교', '소년', '심술', '여자', '심부름']
keyword_list length : 6812

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '로맨스소설', '로맨스', '대공', '소설 속', '유리']
keyword_list length : 6813

list length : 6
pick(cnt) length : 6
keyword : ['스릴

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '죽음', '지젤', '발레', '은지', '스릴러']
keyword_list length : 6874

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '로맨스소설', '국혼', '빙의', '원혼', '폐하']
keyword_list length : 6875

list length : 6
pick(cnt) length : 6
keyword : ['일본소설', '살인사건', '미스터리소설', '소설', '퍼즐', '수수께끼 놀이']
keyword_list length : 6876

list length : 6
pick(cnt) length : 6
keyword : ['영화', '환상문학', '디스토피아', '리세', '소설 전반', '독서']
keyword_list length : 6877

list length : 6
pick(cnt) length : 6
keyword : ['미스터리소설', '살인사건', '약물', '천사', '마약', '남자']
keyword_list length : 6878

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '소설', '로맨스소설', '로맨스', '사랑', '악당']
keyword_list length : 6879

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '소설', '작가', '카카오 페이지', 'SF소설', '작품']
keyword_list length : 6880

list length : 6
pick(cnt) length : 6
keyword : ['한국소설', '사회문제', '노인', '작가', '우체부', '상상력']
keyword_list length : 6881

list length : 6
pick(cnt) length : 6
keyword : [

In [193]:
len(keyword_list)

6903

In [83]:
isbn_K = []

In [84]:
# isbn에 알파벳이 들어가있는 데이터 인덱스 추출 및 출력
for i in range(6911) :
    if 'K' in str(book_remove_df['isbn'][i]) :
        isbn_K.append(i)
        display(book_remove_df[i:i + 1])

,title,author,publisher,genre,isbn,image
1905,뉴필로소퍼 우먼카인드 스켑틱 패키지 1년 정기구독,바다출판사 편집부 (지은이),바다출판사,과학,K502532396,https://image.aladin.co.kr/product/13187/76/co...


,title,author,publisher,genre,isbn,image
2235,한국 스켑틱 1년 정기구독,스켑틱 협회 편집부 (지은이),바다출판사,과학,K022434378,https://image.aladin.co.kr/product/6892/6/cove...


,title,author,publisher,genre,isbn,image
5292,비밀의 정원 (리커버 에디션),조해너 배스포드 (지은이),클,예술/대중문화,K662636870,https://image.aladin.co.kr/product/20828/87/co...


,title,author,publisher,genre,isbn,image
5443,내일은 미스터 트롯 화보집 임영웅 패키지 (임영웅 브로마이드 5장 + 임영웅 포토카...,엔와이컴퍼니 편집부 (지은이),엔와이컴퍼니,예술/대중문화,K000032630267,https://image.aladin.co.kr/product/24245/29/co...


,title,author,publisher,genre,isbn,image
5458,내일은 미스터 트롯 화보집 이찬원 패키지 (이찬원 브로마이드 5장 + 이찬원 포토카...,엔와이컴퍼니 편집부 (지은이),엔와이컴퍼니,예술/대중문화,K000152630267,https://image.aladin.co.kr/product/24245/31/co...


,title,author,publisher,genre,isbn,image
5702,뉴필로소퍼 2020 9호,뉴필로소퍼 편집부 (엮은이),바다출판사,인문학,K632636627,https://image.aladin.co.kr/product/22504/36/co...


,title,author,publisher,genre,isbn,image
5814,뉴필로소퍼 2019 8호,뉴필로소퍼 편집부 (엮은이),바다출판사,인문학,K942636677,https://image.aladin.co.kr/product/21080/41/co...


,title,author,publisher,genre,isbn,image
6012,뉴필로소퍼 2020 10호,뉴필로소퍼 편집부 (엮은이),바다출판사,인문학,K172638348,https://image.aladin.co.kr/product/23700/61/co...


In [85]:
isbn_K

[1905, 2235, 5292, 5443, 5458, 5702, 5814, 6012]

In [92]:
# 인덱스 제거
book_remove_df = book_remove_df.drop(isbn_K)

In [94]:
# 인덱스 제거 후 인덱스 초기화
book_remove_df.reset_index(drop = True, inplace = True)
book_remove_df.tail()

,title,author,publisher,genre,isbn,image
6898,드래곤 펄,"이윤하 (지은이), 송경아 (옮긴이)",사계절,장르소설,9791160946819,https://image.aladin.co.kr/product/25151/52/co...
6899,대표님을 위하여,1그램 (지은이),다향,장르소설,9791165655358,https://image.aladin.co.kr/product/25320/56/co...
6900,해리 포터 영화 속 다이애건 앨리 팝업 가이드 & 그 밖의 장소들 (한국어판 특전 ...,"매슈 라인하트, 조디 리벤슨 (지은이), 케빈 M. 윌슨 (그림), 윤단비 (옮긴이)",문학수첩,장르소설,9788983928191,https://image.aladin.co.kr/product/25399/57/co...
6901,살인범 대 살인귀,"하야사카 야부사카 (지은이), 현정수 (옮긴이)",북로드,장르소설,9791158791476,https://image.aladin.co.kr/product/25447/69/co...
6902,변덕스러운 달,"래리 니븐 (지은이), 이리예 (옮긴이)",새파란상상(파란미디어),장르소설,9788963718026,https://image.aladin.co.kr/product/25373/71/co...


In [197]:
book_remove_df.shape

(6903, 6)

In [198]:
# 데이터프레임에 키워드 데이터 결합
book_remove_df['keyword'] = keyword_list

In [199]:
book_remove_df.head()

,title,author,publisher,genre,isbn,image,keyword
0,트렌드 코리아 2020,"김난도, 전미영, 최지혜, 이향은, 이준영, 김서영, 이수진, 서유현, 권정윤 (지...",미래의창,경제경영,9788959896127,https://image.aladin.co.kr/product/21094/63/co...,"[경제전망, 소비자, 페르소나, 소비, 마우스, 밀레]"
1,린치핀,"세스 고딘 (지은이), 윤영삼 (옮긴이)",라이스메이커,경제경영,9791185392998,https://image.aladin.co.kr/product/22213/37/co...,"[자기관리, 크리에이터, 인공지능, 사업 모델, 톱니바퀴, 예술가]"
2,부자 되는 법을 가르쳐 드립니다,"라밋 세티 (지은이), 김태훈 (옮긴이)",안드로메디안,경제경영,9791135455063,https://image.aladin.co.kr/product/22331/62/co...,"[부자되는법, 재테크, 스크립트, 머니, 돈 관리, 백만장자]"
3,부의 추월차선,"엠제이 드마코 (지은이), 신소영 (옮긴이)",토트,경제경영,9788994702346,https://image.aladin.co.kr/product/3036/43/cov...,"[부자되는법, 부자, 성공법, 백만장자, 퇴직 연금, 구루]"
4,리더의 용기,"브레네 브라운 (지은이), 강주헌 (옮긴이)",갤리온,경제경영,9788901238746,https://image.aladin.co.kr/product/22018/96/co...,"[심리학자, 리더십, 조직관리, 스타트업, 월스트리트 저널, 게이츠]"


In [200]:
# csv로 데이터프레임 저장
book_remove_df.to_csv('book_remove_df.csv', mode = 'w', index = False, encoding = 'utf-8-sig')